In [1]:
import os, sys
import glob
from skimage import transform, io
from skimage.exposure import adjust_gamma
from tqdm import tqdm
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
class random_crop():
    
    def __init__(self, image=None):
        if image is not None:
            self.get_param(image)
        else:
            self.i0 = 0
            self.i1 = 0
            self.j0 = 0
            self.j1 = 0
            
    def get_param(self, image):
        H = image.shape[0]
        W = image.shape[1]
        L = min(H,W)
        size = np.random.randint(int(0.8*L), L)
        self.i0 = np.random.randint(0, H-size)
        self.j0 = np.random.randint(0, W-size)
        self.i1 = self.i0 + size
        self.j1 = self.j0 + size
        
    def __call__(self, image):
        assert(self.i1 < image.shape[0])
        assert(self.j1 < image.shape[1])
        return image[self.i0:self.i1, self.j0:self.j1]
    
    
def process_image_n_mask(image, mask, size=(64,64), flip=False, gamma=1):
    #random crop
    crop = random_crop(image)
    image_processed = crop(image)
    mask_processed = crop(mask)
    #resize
    image_processed = transform.resize(image_processed, size)
    mask_processed = transform.resize(mask_processed, size)
    #flip
    if flip:
        image_processed = np.fliplr(image_processed)
        mask_processed = np.fliplr(mask_processed)
    #addjust gamma
    if gamma != 1:
        image_processed = adjust_gamma(image_processed, gamma)
    return image_processed, mask_processed

def process_image(image, size=(64,64), flip=False, gamma=1):
    #random crop
    crop = random_crop(image)
    image_processed = crop(image)
    #resize
    image_processed = transform.resize(image_processed, size)
    #flip
    if flip:
        image_processed = np.fliplr(image_processed)
    #addjust gamma
    if gamma != 1:
        image_processed = adjust_gamma(image_processed, gamma)
    return image_processed

In [12]:
image_paths = sorted(glob.glob("imagenet/*.JPEG"))
mask_paths = sorted(glob.glob("imagenet_mask/*.jpg"))
mask_path = mask_paths[0]
image_path = os.path.join('imagenet', os.path.split(mask_path)[1][:-4]+'.JPEG')
mask_path, image_path

('imagenet_mask/n03092166_1019.jpg', 'imagenet/n03092166_1019.JPEG')

In [14]:
#imagenet dataset
image_paths = sorted(glob.glob("imagenet/*.JPEG"))
mask_paths = sorted(glob.glob("imagenet_mask/*.jpg"))

!mkdir imagenet_image64
!mkdir imagenet_mask64
for mask_path in tqdm(mask_paths):
    
    image_path = os.path.join('imagenet', os.path.split(mask_path)[1][:-4]+'.JPEG')
    if(not image_path in image_paths):
        continue
    
    image = io.imread(image_path)
    mask = io.imread(mask_path, is_gray=True)
    
    image_processed_path = os.path.join('imagenet_image64', os.path.split(image_path)[1][:-4]+'_1.jpg')
    mask_processed_path = os.path.join('imagenet_mask64', os.path.split(mask_path)[1][:-4]+'_1.jpg')
    image_processed, mask_processed = process_image_n_mask(image, mask, size=(64,64), flip=False)
    io.imsave(image_processed_path, image_processed)
    io.imsave(mask_processed_path, mask_processed)
    
    image_processed_path = os.path.join('imagenet_image64', os.path.split(image_path)[1][:-4]+'_2.jpg')
    mask_processed_path = os.path.join('imagenet_mask64', os.path.split(mask_path)[1][:-4]+'_2.jpg') 
    image_processed, mask_processed = process_image_n_mask(image, mask, size=(64,64), flip=True)
    io.imsave(image_processed_path, image_processed)
    io.imsave(mask_processed_path, mask_processed)    
    
    image_processed_path = os.path.join('imagenet_image64', os.path.split(image_path)[1][:-4]+'_3.jpg')
    mask_processed_path = os.path.join('imagenet_mask64', os.path.split(mask_path)[1][:-4]+'_3.jpg')
    image_processed, mask_processed = process_image_n_mask(image, mask, size=(64,64), flip=False, gamma=0.75)
    io.imsave(image_processed_path, image_processed)
    io.imsave(mask_processed_path, mask_processed)

    image_processed_path = os.path.join('imagenet_image64', os.path.split(image_path)[1][:-4]+'_4.jpg')
    mask_processed_path = os.path.join('imagenet_mask64', os.path.split(mask_path)[1][:-4]+'_4.jpg')
    image_processed, mask_processed = process_image_n_mask(image, mask, size=(64,64), flip=False, gamma=1.25)
    io.imsave(image_processed_path, image_processed)
    io.imsave(mask_processed_path, mask_processed)

mkdir: imagenet_image64: File exists
mkdir: imagenet_mask64: File exists


  0%|          | 0/340 [00:00<?, ?it/s]/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/skimage/transform/_warps.py:105: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/skimage/util/dtype.py:130: UserWarning: Possible precision loss when converting from float64 to uint8
  .format(dtypeobj_in, dtypeobj_out))
 55%|█████▍    | 186/340 [00:00<00:00, 1310.17it/s]/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/skimage/io/_io.py:140: UserWarning: 

In [3]:
#17 flower dataset
image_paths = sorted(glob.glob("oxford17flowers/*.jpg"))
mask_paths = sorted(glob.glob("oxford17flowers_trimap/*.png"))

!mkdir oxford_17flowers_image64
!mkdir oxford_17flowers_mask64
for mask_path in tqdm(mask_paths):
    
    image_path = os.path.join('oxford17flowers', os.path.split(mask_path)[1][:-4]+'.jpg')
    
    image = io.imread(image_path)
    mask = io.imread(mask_path)
    mask = mask[:,:,1]
    mask = (mask<0.1).astype('float')
    
    image_processed_path = os.path.join('oxford_17flowers_image64', os.path.split(image_path)[1][:-4]+'_1.jpg')
    mask_processed_path = os.path.join('oxford_17flowers_mask64', os.path.split(mask_path)[1][:-4]+'_1.jpg')
    image_processed, mask_processed = process_image_n_mask(image, mask, size=(64,64), flip=False)
    io.imsave(image_processed_path, image_processed)
    io.imsave(mask_processed_path, mask_processed)
    
    image_processed_path = os.path.join('oxford_17flowers_image64', os.path.split(image_path)[1][:-4]+'_2.jpg')
    mask_processed_path = os.path.join('oxford_17flowers_mask64', os.path.split(mask_path)[1][:-4]+'_2.jpg') 
    image_processed, mask_processed = process_image_n_mask(image, mask, size=(64,64), flip=True)
    io.imsave(image_processed_path, image_processed)
    io.imsave(mask_processed_path, mask_processed)    
    
    image_processed_path = os.path.join('oxford_17flowers_image64', os.path.split(image_path)[1][:-4]+'_3.jpg')
    mask_processed_path = os.path.join('oxford_17flowers_mask64', os.path.split(mask_path)[1][:-4]+'_3.jpg')
    image_processed, mask_processed = process_image_n_mask(image, mask, size=(64,64), flip=False, gamma=0.75)
    io.imsave(image_processed_path, image_processed)
    io.imsave(mask_processed_path, mask_processed)

    image_processed_path = os.path.join('oxford_17flowers_image64', os.path.split(image_path)[1][:-4]+'_4.jpg')
    mask_processed_path = os.path.join('oxford_17flowers_mask64', os.path.split(mask_path)[1][:-4]+'_4.jpg')
    image_processed, mask_processed = process_image_n_mask(image, mask, size=(64,64), flip=False, gamma=1.25)
    io.imsave(image_processed_path, image_processed)
    io.imsave(mask_processed_path, mask_processed)

  0%|          | 0/846 [00:00<?, ?it/s]/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/skimage/transform/_warps.py:105: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/skimage/util/dtype.py:130: UserWarning: Possible precision loss when converting from float64 to uint8
  .format(dtypeobj_in, dtypeobj_out))
 59%|█████▉    | 501/846 [00:23<00:14, 23.79it/s]/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/skimage/io/_io.py:140: UserWarning: ox

In [15]:
imagenet_paths = glob.glob("imagenet/*.JPEG")
imagenet2_paths = glob.glob("imagenet2/*.JPEG")

!mkdir imagenet64

for image_path in imagenet_paths+imagenet2_paths:# + imagenet2_paths:
    image = io.imread(image_path)
    
    image_processed_path = os.path.join('imagenet64', os.path.split(image_path)[1][:-5]+'_1.jpg')
    image_processed = process_image(image, size=(64,64), flip=False)
    io.imsave(image_processed_path, image_processed)
    
    image_processed_path = os.path.join('imagenet64', os.path.split(image_path)[1][:-5]+'_2.jpg')
    image_processed = process_image(image, size=(64,64), flip=True)
    io.imsave(image_processed_path, image_processed) 
    
    image_processed_path = os.path.join('imagenet64', os.path.split(image_path)[1][:-5]+'_3.jpg')
    image_processed = process_image(image, size=(64,64), flip=False, gamma=0.75)
    io.imsave(image_processed_path, image_processed)

    image_processed_path = os.path.join('imagenet64', os.path.split(image_path)[1][:-5]+'_4.jpg')
    image_processed = process_image(image, size=(64,64), flip=False, gamma=1.25)
    io.imsave(image_processed_path, image_processed)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/skimage/transform/_warps.py:105: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/skimage/util/dtype.py:130: UserWarning: Possible precision loss when converting from float64 to uint8
  .format(dtypeobj_in, dtypeobj_out))


In [8]:
imagenet_nonflower_paths = glob.glob("imagenet_hellebore/*.JPEG")
imagenet_nonflower_paths += glob.glob("imagenet_poison/*.JPEG")
imagenet_nonflower_paths += glob.glob("imagenet_poison2/*.JPEG")
imagenet_nonflower_paths += glob.glob("imagenet_solomen/*.JPEG")
imagenet_nonflower_paths += glob.glob("imagenet_golden/*.JPEG")
imagenet_nonflower_paths += glob.glob("imagenet_galingale/*.JPEG")
imagenet_nonflower_paths += glob.glob("imagenet_umbrella/*.JPEG")
imagenet_nonflower_paths += glob.glob("imagenet_aralia/*.JPEG")
imagenet_nonflower_paths += glob.glob("imagenet_western_ragweed/*.JPEG")
imagenet_nonflower_paths += glob.glob("imagenet_great_ragweed/*.JPEG")
imagenet_nonflower_paths += glob.glob("imagenet_wood_nettle/*.JPEG")
imagenet_nonflower_paths += glob.glob("imagenet_wild_parsnip/*.JPEG")
imagenet_nonflower_paths += glob.glob("imagenet_sting/*.JPEG")
imagenet_nonflower_paths += glob.glob("imagenet_rattlesnake/*.JPEG")
imagenet_nonflower_paths += glob.glob("imagenet_cryptogam/*.JPEG")
imagenet_nonflower_paths += glob.glob("imagenet_hemlock/*.JPEG")

!mkdir nonflower64

for image_path in tqdm(imagenet_nonflower_paths):# + imagenet2_paths:
    image = io.imread(image_path)
    
    image_processed_path = os.path.join('nonflower64', os.path.split(image_path)[1][:-5]+'_1.jpg')
    image_processed = process_image(image, size=(64,64), flip=False)
    io.imsave(image_processed_path, image_processed)
    
    image_processed_path = os.path.join('nonflower64', os.path.split(image_path)[1][:-5]+'_2.jpg')
    image_processed = process_image(image, size=(64,64), flip=True)
    io.imsave(image_processed_path, image_processed)

mkdir: nonflower64: File exists



  0%|          | 0/3885 [00:00<?, ?it/s]/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/skimage/transform/_warps.py:105: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/skimage/util/dtype.py:130: UserWarning: Possible precision loss when converting from float64 to uint8
  .format(dtypeobj_in, dtypeobj_out))

 40%|████      | 1561/3885 [00:48<00:55, 41.82it/s]


 96%|█████████▌| 3713/3885 [01:27<00:02, 65.04it/s]


100%|██████████| 3885/3885 [01:30<00:00, 42.99it/s]